In [31]:
Liam = ((38*2)/6)
Liam = round(Liam, 2)
Liam

12.67

In [33]:
Ioan = (((38*2)+40)/6) + 20/6
Ioan = round(Ioan, 2)
Ioan

22.67

In [16]:
Bailey = 40/6
Bailey

6.666666666666667

In [34]:
Morgan = Ioan
Morgan

22.67

In [35]:
Rhys = Ioan
Rhys

22.67

In [36]:
Beth = Ioan
Beth

22.67

In [37]:
Dan = Ioan

In [41]:
total = Dan + Beth + Rhys + Ioan + Liam  + Morgan
total

126.02000000000001

In [40]:
print(f"Dan: £{Dan}, Beth: £{Beth}, Rhys: £{Rhys}, Ioan: £{Ioan}, Liam: £{Liam}, Morgan: £{Morgan}")

Dan: £22.67, Beth: £22.67, Rhys: £22.67, Ioan: £22.67, Liam: £12.67, Morgan: £22.67


In [ ]:
    def anneal_NVT(self, simulation, start_temp=None, max_temp=None, cycles=None, quench_rate=None, steps_per_cycle=None):
        """
        Function to perform simulated annealing on the provided simulation system.
        
        USAGE: 
            annealed_sim = sim.anneal(simulation, start_temp, max_temp, cycles, holding_steps, steps_at_temp)            
        
        Recommended USAGE:
            annealed_sim = sim.anneal(simulation)
            
            where annealing parameters are set with the following function:
                
            simulation_object.set_anneal_parameters([start_temp, max_temp, cycles, holding_steps, steps_at_each_temp])
        
        Args:
            simulation (app.Simulation): The simulation object to perform annealing on.
            
            start_temp (float, optional): The starting temperature for annealing in Kelvin. Defaults to None, 
                in which case the value is fetched from self.anneal_parameters[0].
                
            max_temp (float, optional): The maximum temperature for annealing in Kelvin. Defaults to None, 
                in which case the value is fetched from self.anneal_parameters[1].
                
            cycles (int, optional): The number of annealing cycles to perform. Defaults to None, 
                in which case the value is fetched from self.anneal_parameters[2].
                
            holding_steps (int, optional): The number of steps to hold the system at each temperature. 
                Defaults to None, in which case the value is fetched from self.anneal_parameters[3].
                
            steps_at_temp (int, optional): The number of steps to perform at each temperature. 
                Defaults to None, in which case the value is fetched from self.anneal_parameters[4].

        Returns:
            A tuple containing the simulation object after annealing and the filename of the data file generated.
                in the following format:
        
            (simulation_object, path_to_data_file)
                    
        Notes:
            This method performs simulated annealing on the provided simulation object. It initializes the system with the 
        provided initial conditions and runs annealing cycles adjusting the temperature according to the specified parameters.
        The system's state is updated throughout the annealing process, and reporters are set up to record trajectory and 
        simulation data.
        """
        anneal_start_time = time.time()
        
        if start_temp is None:
            start_temp = self.anneal_parameters[0]
        if max_temp is None:
            max_temp = self.anneal_parameters[1]
        if cycles is None:
            cycles = self.anneal_parameters[2]
        if quench_rate is None:
            holding_steps = self.anneal_parameters[3]
        if steps_per_cycle is None:
            steps_at_temp = self.anneal_parameters[4]
         
        # Extract positional info
        state = simulation.context.getState(getPositions=True, getEnergy=True, enforcePeriodicBox=True) # Define state object 
        xyz = state.getPositions() # Obtain positions of the particles from previous step
        vx, vy, vz = state.getPeriodicBoxVectors() # Obtain periodc box vectors of the previous step
        
        # Set up integrator
        integrator = LangevinIntegrator(start_temp*kelvin, self.friction_coeff/picoseconds, self.timestep*femtoseconds)
        
        if BuildSimulation.type_of_simulation(self) == "AMB":
            system = self.amb_topology.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=self.nonbondedcutoff*nanometers, constraints=app.HBonds)
            simulation = app.Simulation(self.amb_topology.topology, system, integrator)
        
        if BuildSimulation.type_of_simulation(self) == "ANI":
            system = self.potential.createSystem(self.ani_topology, nonbondedMethod=app.PME, nonbondedCutoff=self.nonbondedcutoff*nanometers, constraints=app.HBonds)
            platform = PLatform.getPlatformByName('CUDA')
            simulation = app.Simulation(self.ani_topology, system, integrator, platform)

        # Update the xyz of each atom
        simulation.context.setPeriodicBoxVectors(vx, vy, vz)
        simulation.context.setPositions(xyz)
        
        # Total up all steps of the equilibration so the rpeorters record properly
        total_steps = steps_per_cycle*cycles
        
        # Set up reporters
        # PDB trajectory - this is slighlty redundant with the addition of the DCD trajectory, but it is still useful for 
        if self.savepdb_traj == True:
            output_pdbname = os.path.join(self.output_dir, (self.filename + "_anneal_" + str(self.timestamp) + ".pdb" ))
            simulation.reporters.append(app.PDBReporter(output_pdbname, self.reporter_freq))
        
        # DCD trajectory
        #output_dcdname = os.path.join(directories.systems_dir, self.filename, (self.filename + "_anneal"))
        output_dcdname = os.path.join(self.output_dir, (self.filename + "_anneal_" + str(self.timestamp)))
        dcdWriter = DcdWriter(output_dcdname, self.reporter_freq)
        simulation.reporters.append(dcdWriter.dcdReporter)
    
        # Datawriter - This is a more complete data writer than previously used, the file generated is a comma delimited text file
        #output_dataname = os.path.join(directories.systems_dir, self.filename, (self.filename + "_anneal_data"))
        output_dataname = os.path.join(self.output_dir, (self.filename + "_anneal_" + str(self.timestamp)))
        dataWriter = DataWriter(output_dataname, self.reporter_freq, total_steps)
        simulation.reporters.append(dataWriter.stateDataReporter)
        
        increments = int(max_temp-start_temp)/quench_rate
        steps_per_slope = int(steps_per_cycle*0.4)
        holding_steps = int(steps_per_cycle*0.1) 
        steps_at_increment = int(steps_per_slope/increments)

        def cycle(start_temp, max_temp, steps_at_increment, holding_steps, increments, quench_rate):
            integrator.setTemperature(start_temp)
            simulation.step(steps_at_increment)

            for i in range(increments):
                integrator.setTemperature(start_temp + (i*quench_rate))
                simulation.step(steps_at_increment)

            integrator.setTemperature(max_temp)
            simulation.step(holding_steps)
            
            for i in range(increments):
                integrator.setTemperature(max_temp - (i*quench_rate))
                simulation.step(steps_at_increment)                

            integrator.setTemperature(start_temp)
            simulation.step(holding_steps)
                
        for i in range(cycles):
            cycle(start_temp, max_temp, steps_at_increment, holding_steps, increments, quench_rate)
            
        anneal_end_time = time.time()  
        time_taken = anneal_end_time - anneal_start_time

        self.log_info['Annealing_NVT']['Time taken'] = time_taken
        self.log_info['Annealing_NVT']['Simulation time'] = total_steps*self.timestep
        self.log_info['Annealing_NVT']['Start temp'] = start_temp
        self.log_info['Annealing_NVT']['Target temp'] = max_temp
        self.log_info['Annealing_NVT']['Cycles'] = cycles
        self.log_info['Annealing_NVT']['Steps at plateaus'] = holding_steps
        self.log_info['Annealing_NVT']['Steps at incremental temps'] = steps_at_temp
        self.log_info['Annealing_NVT']['Timestep'] = self.timestep

        # Write the final structure to pdb
        self.final_pdbname = os.path.join(self.output_dir, ("final_anneal_" + self.filename + ".pdb"))
        with open(self.final_pdbname, 'w') as output:
            PDBFile.writeFile(simulation.topology, state.getPositions(), output)
        
        return(simulation, (output_dataname + ".txt"))

In [ ]:
    @classmethod
    def set_anneal_parameters(cls, new_anneal_parameters): 
        """
        Class method to set annealing parameters.

        Args:
            cls: The class itself.
            new_anneal_parameters (list): List of annealing parameters in the format [start_temp, max_temp, cycles, holding_steps, steps_at_temp].

        Returns:
            None

        Raises:
            ValueError: If the length of new_anneal_parameters does not match the expected length.

        Notes:
            This method sets the annealing parameters class attribute to the specified list. 
            It prints a confirmation message with the provided parameters.
        """
        if len(new_anneal_parameters) != len(cls.anneal_parameters):
            format_str = "Expected format: [start_temp, max_temp, cycles, quench_rate, steps_per_cycle]"
            raise ValueError(f"Invalid parameters provided. {format_str}")
        else: 
            # new_anneal_parameters = [start_temp, max_temp, cycles, holding_steps]
            cls.anneal_parameters = new_anneal_parameters
            print("Anneal parameters set.")
            print("Starting temperature is: ", str(new_anneal_parameters[0]))
            print("Target temperature is: ", str(new_anneal_parameters[1]))
            print("Number of annealing cycles is: ", str(new_anneal_parameters[2]))
            print("The quench rate is: ", str(new_anneal_parameters[3]))
            print("The number of steps per cycle is: ", str(new_anneal_parameters[4]))